In [7]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


In [8]:
!pip install -q pyyaml h5py

In [9]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [10]:
# 간단한 Sequential 모델을 정의합니다
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

    return model

# 모델 객체를 만듭니다
model = create_model()

# 모델 구조를 출력합니다
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# 훈련 하는 동안 체크포인트 저장

In [11]:
checkpoint_path = "/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 1s 8ms/step - loss: 1.1599 - accuracy: 0.6720 - val_loss: 0.7178 - val_accuracy: 0.7810

Epoch 00001: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4226 - accuracy: 0.8700 - val_loss: 0.5597 - val_accuracy: 0.8100

Epoch 00002: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2939 - accuracy: 0.9210 - val_loss: 0.4921 - val_accuracy: 0.8410

Epoch 00003: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2192 - accuracy: 0.9480 - val_loss: 0.4630 - val_accuracy: 0.8590

Epoch 00004: saving model to /content/drive/MyDrive/Study/01_tensorflow/T

In [12]:
ls {checkpoint_dir}

'제목없는 폴더'/  '제목없는 폴더.data-00000-of-00001'
 checkpoint       '제목없는 폴더.index'


In [14]:
# 기본 모델 객체를 만듭니다
model = create_model()

# 모델을 평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3716 - accuracy: 0.0700
훈련되지 않은 모델의 정확도:  7.00%


In [15]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4088 - accuracy: 0.8700
복원된 모델의 정확도: 87.00%


# 체크포인트 콜백 매개변수

In [16]:

# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0005.ckpt

Epoch 00010: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0010.ckpt

Epoch 00015: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0015.ckpt

Epoch 00020: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0020.ckpt

Epoch 00025: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0025.ckpt

Epoch 00030: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0030.ckpt

Epoch 00035: saving model to /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0035.ckpt

Epoch 00040: saving model to /content/drive/MyDrive/St

In [17]:
ls {checkpoint_dir}

ls: cannot access '/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는': No such file or directory
ls: cannot access '폴더': No such file or directory


In [18]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/checkpoint/제목없는 폴더/cp-0050.ckpt'

In [19]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4938 - accuracy: 0.8760
복원된 모델의 정확도: 87.60%


In [23]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model_save_path = "/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/model_save/my_model"
model.save(model_save_path)

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1573 - accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4208 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2805 - accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2042 - accuracy: 0.9450
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1601 - accuracy: 0.9600
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/model_save/my_model/assets


In [24]:
new_model = tf.keras.models.load_model(model_save_path)

# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4465 - accuracy: 0.8580
복원된 모델의 정확도: 85.80%
(1000, 10)


# HDF5 파일로 저장하기

In [27]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model_save_HDF5_path = "/content/drive/MyDrive/Study/01_tensorflow/Tensorflow_tutorial/model_save_HDF5/my_model.h5"
model.save(model_save_HDF5_path)

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1471 - accuracy: 0.6730
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4162 - accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2911 - accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2038 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1426 - accuracy: 0.9700


In [29]:
new_model = tf.keras.models.load_model(model_save_HDF5_path)

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4272 - accuracy: 0.8690
복원된 모델의 정확도: 86.90%
